In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "movie": "파묘",
        "answer": """
        감독: 장재현
        주요 출연진: 최민식, 김고은, 유해진, 이도현
        예산: 140억원
        흥행 수익: 약 1300억원
        장르: 미스터리, 공포
        시놉시스: 미국 LA, 거액의 의뢰를 받은 무당 ‘화림’(김고은)과 ‘봉길’(이도현)은 기이한 병이 대물림되는 집안의 장손을 만난다. 조상의 묫자리가 화근임을 알아챈 ‘화림’은 이장을 권하고, 돈 냄새를 맡은 최고의 풍수사 ‘상덕’(최민식)과 장의사 ‘영근’(유해진)이 합류한다. “전부 잘 알 거야… 묘 하나 잘못 건들면 어떻게 되는지” 절대 사람이 묻힐 수 없는 악지에 자리한 기이한 묘. ‘상덕’은 불길한 기운을 느끼고 제안을 거절하지만, ‘화림’의 설득으로 결국 파묘가 시작되고… 나와서는 안될 것이 나왔다.
        """,
    },
    {
        "movie": "서울의 봄",
        "answer": """
        감독: 김성수
        주요 출연진: 황정민, 정우성, 이성민, 박해준, 김성균
        예산: 232.5억원
        흥행 수익: 약 1300억원
        장르: 드라마
        시놉시스: 1979년 12월 12일, 수도 서울 군사반란 발생 그날, 대한민국의 운명이 바뀌었다 대한민국을 뒤흔든 10월 26일 이후, 서울에 새로운 바람이 불어온 것도 잠시 12월 12일, 보안사령관 전두광이 반란을 일으키고 군 내 사조직을 총동원하여 최전선의 전방부대까지 서울로 불러들인다. 권력에 눈이 먼 전두광의 반란군과 이에 맞선 수도경비사령관 이태신을 비롯한 진압군 사이, 일촉즉발의 9시간이 흘러가는데… 목숨을 건 두 세력의 팽팽한 대립 오늘 밤, 대한민국 수도에서 가장 치열한 전쟁이 펼쳐진다!
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "영화 {movie}에 대해서 아는 것을 말해주세요."),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 영화 전문가입니다. 간략한 내용으로 답해주세요."),
        example_prompt,
        ("human", "영화 {movie}에 대해서 아는 것을 말해주세요."),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "택시운전사"})


        감독: 장훈
        주요 출연진: 송강호, 유해진
        예산: 약 100억원
        흥행 수익: 약 1200억원
        장르: 드라마, 코미디
        시놉시스: 1980년대 서울을 배경으로 한 택시 기사 ‘만석’(송강호)이 독일인 기자 ‘피터’(토마스 크레취만)를 태우고, 그의 요청에 따라 1980년 5월의 민주화 운동과 관련된 이야기를 듣게 된다. 피터는 당시의 상황을 기록하기 위해 택시를 이용하며, 만석은 그의 이야기를 듣고 자신의 과거와 현재에 대해 다시 한번 돌아보게 된다. 이를 통해 역사적 사실과 인간적 감정이 어우러진 감동적인 이야기가 전개된다.

AIMessageChunk(content='\n        감독: 장훈\n        주요 출연진: 송강호, 유해진\n        예산: 약 100억원\n        흥행 수익: 약 1200억원\n        장르: 드라마, 코미디\n        시놉시스: 1980년대 서울을 배경으로 한 택시 기사 ‘만석’(송강호)이 독일인 기자 ‘피터’(토마스 크레취만)를 태우고, 그의 요청에 따라 1980년 5월의 민주화 운동과 관련된 이야기를 듣게 된다. 피터는 당시의 상황을 기록하기 위해 택시를 이용하며, 만석은 그의 이야기를 듣고 자신의 과거와 현재에 대해 다시 한번 돌아보게 된다. 이를 통해 역사적 사실과 인간적 감정이 어우러진 감동적인 이야기가 전개된다.')